# Evaluation Scenarios

In [1]:
using Random
using Printf
using StaticArrays

# POMDP and learning
using POMDPs
using BeliefUpdaters
using POMDPPolicies
using POMDPSimulators
using POMDPModelTools
using LocalApproximationValueIteration
using RLInterface
using DeepQLearning
using Flux

# Driving related Packages
using AutomotiveDrivingModels
using AutomotiveSensors
using AutomotivePOMDPs
using PedCar

# Visualization
using AutoViz
set_color_theme(LIGHTTHEME)
AutoViz._colortheme["background"] = colorant"white"
AutoViz._colortheme["CROSSWALK"] = colorant"gray"
using Reel
using ProgressMeter

┌ Info: Precompiling AutomotiveSensors [9ec66b5e-c605-11e8-1454-ede748df6f96]
└ @ Base loading.jl:1273
┌ Info: Precompiling AutomotivePOMDPs [dca8afc0-c686-11e8-0934-712707da73aa]
└ @ Base loading.jl:1273
┌ Info: Precompiling PedCar [90cf7f26-d5c7-593d-a0e1-4a8367407571]
└ @ Base loading.jl:1273
  ** incremental compilation may be fatally broken for this module **



In [2]:
include("../src/masking.jl")
include("../src/util.jl")
include("../src/masked_dqn.jl")
include("../src/qmdp_approximation.jl")
include("../src/render_helpers.jl")

animate_history (generic function with 1 method)

## Environment parameters

In [3]:
rng = MersenneTwister(1);
cam =StaticCamera(VecE2(0., -8.), 14.0);

In [4]:
params = UrbanParams(nlanes_main=1,
                     crosswalk_pos =[VecSE2(6, 0., pi/2), VecSE2(-6, 0., pi/2), VecSE2(0., -5., 0.)],
                     crosswalk_length =  [14.0, 14., 14.0],
                     crosswalk_width = [4.0, 4.0, 3.1],
                     stop_line = 22.0)
env = UrbanEnv(params=params);

In [5]:
pomdp = UrbanPOMDP(env=env,
                   sensor = PerfectSensor(),
                   ego_goal = LaneTag(2, 1),
                   max_cars=4, 
                   max_peds=4, 
                   car_birth=0., 
                   ped_birth=0., 
                   max_obstacles=0, # no fixed obstacles
                   lidar=false,
                   ego_start=20,
                   ΔT=0.1);

In [6]:
const TURN_RIGHT = SVector(LaneTag(3,1), LaneTag(5,1))
const STRAIGHT_FROM_RIGHT = SVector(LaneTag(3,1), LaneTag(4,1))
const STRAIGHT_FROM_LEFT = SVector(LaneTag(1,1), LaneTag(2,1))
const TURN_LEFT = SVector(LaneTag(1,1), LaneTag(5,1));

In [7]:
const RIGHT_OBSTACLE = ConvexPolygon([VecE2(8.125, -7.500), VecE2(26.875, -7.500), VecE2(26.875, -3.000), VecE2(8.125, -3.000)], 4)
const LEFT_OBSTACLE = ConvexPolygon([VecE2(-26.875, -7.500),VecE2(-8.125, -7.500),VecE2(-8.125, -3.000),VecE2(-26.875, -3.000)], 4);

## Evaluation Policy

Choose the belief updater and the policy to evaluate. 

Default: always choose 0. (stays still) and no belief update.

In [8]:

# Set belief updater
updater = NothingUpdater()

# Set policy
policy = FunctionPolicy(s -> UrbanAction(0.)) # constant policy


FunctionPolicy{var"#93#94"}(var"#93#94"())

## Case study 1: Perception Error

It is difficult to handle perception error in rule-based policies. The POMDP framework, through the use of belief state, is well suited to address perception errors like sensor noise, false positive, false negative or sensor occlusion. 

### Scenario 1.1: Make some noise!

Regular intersection, with cars and pedestrian, no obstacles. We are only looking at how sensor noise is taken care of.

*Problem variables:*
- car starting position and velocity
- car route (straight, turn right, turn left)
- car presence 
- pedestrian starting position, crosswalk, and velocity
- pedestrian presence
- sensor characteristics

In [9]:
# Set car parameter
car_on = true
car_s0 = 0.
car_v0 = 0.
car_posF = Frenet(env.roadway[LaneTag(3, 1)], car_s0)
car = Vehicle(VehicleState(car_posF, env.roadway, car_v0), pomdp.car_type, 2)

# Set pedestrian parameter
ped_on = false
ped_s0 = 0.
ped_v0 = 0.
ped_posF = Frenet(env.ped_roadway[LaneTag(18,1)], ped_s0) # choose between 17, 18, 19
ped = Vehicle(VehicleState(ped_posF, env.roadway, ped_v0), pomdp.ped_type, 101)

# Set sensor characteristics
pomdp.sensor = GaussianSensor(pos_noise = LinearNoise(min_noise=0.5, increase_rate=0.1),
                              vel_noise = LinearNoise(min_noise=0.5, increase_rate=0.1),
                              false_positive_rate = 0.1,
                              false_negative_rate = 0.1)

# Initial state
function initialize_scenario(pomdp::UrbanPOMDP, car::Vehicle, ped::Vehicle, car_route; car_on=true, ped_on=true)
    s0 = Scene()

    if car_on
        push!(s0, car)
        pomdp.max_cars = 1
        pomdp.models[2] = pomdp.car_models[car_route]
    end
    if ped_on
        push!(s0, ped)
        pomdp.max_peds = 1
        pomdp.models[101] = IntelligentPedestrian(dt = pomdp.ΔT, crosswalk=get_lane(env.roadway, ped), conflict_lanes=get_conflict_lanes(get_lane(env.roadway, ped), env.roadway))
    end

    push!(s0, initial_ego(pomdp, rng))
    return s0
end
s0 = initialize_scenario(pomdp, car, ped, STRAIGHT_FROM_RIGHT, car_on=car_on, ped_on=ped_on)

# Static scenario specs
empty_obstacles!(env)
pomdp.ped_birth = 0.
pomdp.car_birth = 0.


# Initial belief 
b0 = nothing
hr = HistoryRecorder(max_steps=100, rng=rng)
@time hist = POMDPSimulators.simulate(hr, pomdp, policy, updater, b0, s0);

  8.420608 seconds (15.53 M allocations: 784.314 MiB, 3.98% gc time)


In [10]:
animate_history(hist, pomdp,
                obs_overlays = o -> [GaussianSensorOverlay(sensor=pomdp.sensor, o=[veh for veh in obs_to_scene(pomdp, o) if veh.id != EGO_ID], color=MONOKAY["color2"])],
                step_overlays = s -> [TextOverlay(text = ["step: $s"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3.,4.), incameraframe=true)],
                extra_overlays = [IDOverlay()],
                speed_factor = 2,
                cam =  cam)

Frames{MIME{Symbol("image/png")}}("C:\\Users\\Maxime\\AppData\\Local\\Temp\\jl_wbNuNo", 0x0000000000000064, 20.0, nothing)

### Scenario 1.2: Blind Corners

A similar scenario than above, with sensor occlusions. 

*Problem variables:*
- car starting position
- car route (straight, turn right, turn left)
- car presence 
- pedestrian starting position and crosswalk 
- pedestrian presence
- sensor characteristics
- obstacle location: right or left

In [11]:
# set obstacles
pomdp.max_obstacles = 1
env.obstacles = [RIGHT_OBSTACLE]

# set car position
car_on = true
car_s0 = 0.
car_v0 = 0.
car_posF = Frenet(env.roadway[LaneTag(1, 1)], car_s0)
car = Vehicle(VehicleState(car_posF, env.roadway, car_v0), pomdp.car_type, 2)

s0 = initialize_scenario(pomdp, car, ped, TURN_LEFT, ped_on=false)
# Initial belief 
b0 = nothing

hr = HistoryRecorder(max_steps=100, rng=rng)
@time hist = POMDPSimulators.simulate(hr, pomdp, policy, updater, b0, s0);

  0.067645 seconds (49.48 k allocations: 4.923 MiB)


In [12]:
animate_history(hist, pomdp,
                state_overlays = s -> [BlinkerOverlay(right=false, on=true, veh=v) for v in s if v.id==2],
                obs_overlays = o -> [GaussianSensorOverlay(sensor=GaussianSensor(), o=[veh for veh in obs_to_scene(pomdp, o) if veh.id != EGO_ID], color=AutoViz.MONOKAY["color2"])],
                step_overlays = s -> [TextOverlay(text = ["step: $s"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3.,4.), incameraframe=true)],
                extra_overlays = [AutoViz.IDOverlay()],
                speed_factor = 2,
                cam =  cam)

Frames{MIME{Symbol("image/png")}}("C:\\Users\\Maxime\\AppData\\Local\\Temp\\jl_DNNJ0i", 0x0000000000000064, 20.0, nothing)

## Case Study 2: Interactions

The POMDP approach can capture interaction between other traffic participants such as a vehicle yielding for a pedestrian or letting the right of way to our vehicle

### Scenario 2.1: Vehicle - Pedestrian interaction

*Problem variables:*
- car starting position
- car route (straight, turn right, turn left)
- car presence 
- pedestrian starting position and crosswalk 
- pedestrian presence

In [14]:
# Set car parameters
car_on = true
car_s0 = 0.
car_v0 = 0.
car_posF = Frenet(env.roadway[LaneTag(1, 1)], car_s0)
car = Vehicle(VehicleState(car_posF, env.roadway, car_v0), pomdp.car_type, 2)

# Set pedestrian parameters
ped_on = true
ped_s0 = 2.
ped_v0 = 0.
ped_posF = Frenet(env.ped_roadway[LaneTag(17,1)], ped_s0) # choose between 17, 18, 19
ped = Vehicle(VehicleState(ped_posF, env.roadway, ped_v0), pomdp.ped_type, 101)

# Sensor 
pomdp.sensor = PerfectSensor()
empty_obstacles!(pomdp.env)

s0 = initialize_scenario(pomdp, car, ped, STRAIGHT_FROM_RIGHT)

# Initial belief 
b0 = nothing

hr = HistoryRecorder(max_steps=100, rng=rng)
@time hist = POMDPSimulators.simulate(hr, pomdp, policy, updater, b0, s0);

  0.034588 seconds (46.87 k allocations: 4.159 MiB)


In [15]:
animate_history(hist, pomdp,
                step_overlays = s -> [TextOverlay(text = ["step: $s"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3.,4.), incameraframe=true)],
                extra_overlays = [IDOverlay()],
                speed_factor = 2,
                cam =  cam)

Frames{MIME{Symbol("image/png")}}("C:\\Users\\Maxime\\AppData\\Local\\Temp\\jl_eOvptq", 0x0000000000000064, 20.0, nothing)

### Scenario 2.2: Vehicle - Vehicle interactions

I am currently unsure if it is relevant. 

A possible situation is one where the ego vehicle is starting the turn while a car on the right arrives, intending to make a left turn and is then going to let the priority to the ego vehicle.

In [16]:
# Set car parameter
car_s0 = 10.
car_v0 = 0.
car_posF = Frenet(env.roadway[LaneTag(1, 1)], car_s0)
car = Vehicle(VehicleState(car_posF, env.roadway, car_v0), pomdp.car_type, 2)

# Sensor 
pomdp.sensor = PerfectSensor()

s0 = initialize_scenario(pomdp, car, ped, TURN_LEFT, ped_on=false)

# Initial belief 
b0 = nothing

# Change the policy
slow_policy = FunctionPolicy(x->UrbanAction(0.5))

hr = HistoryRecorder(max_steps=50, rng=rng)
@time hist = POMDPSimulators.simulate(hr, pomdp, slow_policy, updater, b0, s0);

  1.630519 seconds (229.69 k allocations: 34.558 MiB, 1.92% gc time)


In [17]:
animate_history(hist, pomdp,
                state_overlays = s -> [BlinkerOverlay(right=false, on=true, veh=v) for v in s if v.id==2],
                step_overlays = s -> [TextOverlay(text = ["step: $s"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3.,4.), incameraframe=true)],
                extra_overlays = [IDOverlay()],
                speed_factor = 2,
                cam =  cam)

Frames{MIME{Symbol("image/png")}}("C:\\Users\\Maxime\\AppData\\Local\\Temp\\jl_ALKUQg", 0x0000000000000032, 20.0, nothing)

## Case Study 3: Scalability

Our approach scales to multiple cars, pedestrians and obstacles. Conventional rule-based approach are very difficult to design in such scenarios since there are a lot of cases to take into account.

*Problem variables:*
- Maximum number of cars 
- Maximum number of pedestrian 
- Probability of appearance of cars 
- Probability of appearance of pedestrians

In [18]:
pomdp.max_cars = 3.
pomdp.max_peds = 3.
pomdp.car_birth = 0.3
pomdp.ped_birth = 0.3
pomdp.max_obstacles = 0.

s0 = initialstate(pomdp, rng)

# Initial belief 
b0 = nothing

hr = HistoryRecorder(max_steps=200, rng=rng)
@time hist = POMDPSimulators.simulate(hr, pomdp, policy, updater, b0, s0);

  0.149581 seconds (192.44 k allocations: 13.475 MiB)


In [19]:
animate_history(hist, pomdp,
                step_overlays = s -> [TextOverlay(text = ["step: $s"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3.,4.), incameraframe=true)],
                extra_overlays = [IDOverlay()],
                speed_factor = 2,
                cam =  cam)

Frames{MIME{Symbol("image/png")}}("C:\\Users\\Maxime\\AppData\\Local\\Temp\\jl_Sz4Aze", 0x00000000000000c8, 20.0, nothing)